# Task 1.3 – ECG - data augmentation and feature engineering 
Augment the data and featurer engineer with

1.3 Data Augmentation and Feature Engineering 

In [1]:
import pickle
from collections import Counter

with open("../data/split_data.pkl", "rb") as f:
    X_train_split, X_val_split, y_train_split, y_val_split = pickle.load(f)

print(f"Trainingsdaten: {len(X_train_split)} Zeitreihen")
print(f"Validierungsmenge: {len(X_val_split)} Beispiele")
print("Train-class-distribution:", Counter(y_train_split))
print("Val-class-distribution:", Counter(y_val_split))


Trainingsdaten: 4943 Zeitreihen
Validierungsmenge: 1236 Beispiele
Train-class-distribution: Counter({0: 2910, 2: 1412, 1: 439, 3: 182})
Val-class-distribution: Counter({0: 728, 2: 353, 1: 110, 3: 45})


In [2]:
import torch
import torch.nn.functional as F

class STFTLayer(torch.nn.Module):
    def __init__(self, n_fft=64, hop_length=16):
        super().__init__()
        self.n_fft = n_fft
        self.hop_length = hop_length

    def forward(self, x):
        stft_tensors = []
        max_time_steps = 0

        # First pass: compute STFTs and find max time dimension
        for signal in x:
            signal_tensor = torch.tensor(signal, dtype=torch.float32)
            stft = torch.stft(signal_tensor, n_fft=self.n_fft, hop_length=self.hop_length,
                              return_complex=True)
            magnitude = stft.abs()
            stft_tensors.append(magnitude)
            max_time_steps = max(max_time_steps, magnitude.shape[1])

        # Second pass: pad all to the same shape
        padded = []
        for m in stft_tensors:
            pad_size = max_time_steps - m.shape[1]
            m_padded = F.pad(m, (0, pad_size))  # pad last dim (time) to the right
            padded.append(m_padded)

        return torch.stack(padded)
    
stft_layer = STFTLayer(n_fft=64, hop_length=16)
X_train_stft = stft_layer(X_train_split)
X_val_stft = stft_layer(X_val_split)

y_train = y_train_split
y_val = y_val_split

print(X_train_stft.shape)
print(X_val_stft.shape)


/Users/mfsafi/Desktop/AMLS Project/ehsan_AMLS_Exercise/.venv/lib/python3.9/site-packages/torch/functional.py:730: UserWarning: A window was not provided. A rectangular window will be applied,which is known to cause spectral leakage. Other windows such as torch.hann_window or torch.hamming_window are recommended to reduce spectral leakage.To suppress this warning and use a rectangular window, explicitly set `window=torch.ones(n_fft, device=<device>)`. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/SpectralOps.cpp:842.)
  return _VF.stft(  # type: ignore[attr-defined]


torch.Size([4943, 33, 1142])
torch.Size([1236, 33, 1143])


In [3]:
import sys
sys.path.append("../1.1_dataset_exploration/src")  # adjust path if needed
from dataset import create_spectrogram_dataloaders

# Create DataLoaders with augmentations for training
train_loader, val_loader = create_spectrogram_dataloaders(
    X_train_stft, y_train, X_val_stft, y_val, batch_size=32, augment=True
)

# Check one batch (optional)
for xb, yb in train_loader:
    print("Batch shape:", xb.shape)
    break

Batch shape: torch.Size([32, 1, 33, 1142])


In [ ]:
import torch
import torch.nn as nn

# Import your model from model.py (adjust path if needed)
from model import ECGCNN

# Define the training function
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs = model(X_val)
                loss = criterion(outputs, y_val)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += y_val.size(0)
                val_correct += (predicted == y_val).sum().item()

        val_acc = val_correct / val_total

        print(f"Epoch {epoch+1}: "
              f"Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
              f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

# Instantiate and train the model
cnn_model = ECGCNN()
train_model(cnn_model, train_loader, val_loader, epochs=100)

Epoch 1: Train Loss=200.1508, Train Acc=0.5141, Val Loss=47.3291, Val Acc=0.5890
Epoch 2: Train Loss=186.0347, Train Acc=0.5642, Val Loss=44.2877, Val Acc=0.5890
Epoch 3: Train Loss=176.2215, Train Acc=0.5675, Val Loss=42.4021, Val Acc=0.5890
Epoch 4: Train Loss=172.3179, Train Acc=0.5735, Val Loss=41.2844, Val Acc=0.5890
Epoch 5: Train Loss=170.8521, Train Acc=0.5713, Val Loss=40.9420, Val Acc=0.5914
Epoch 6: Train Loss=167.8655, Train Acc=0.5733, Val Loss=40.2210, Val Acc=0.5906
Epoch 7: Train Loss=167.0938, Train Acc=0.5774, Val Loss=40.2273, Val Acc=0.5906
Epoch 8: Train Loss=166.8392, Train Acc=0.5719, Val Loss=40.3340, Val Acc=0.5914
Epoch 9: Train Loss=165.7460, Train Acc=0.5733, Val Loss=40.0386, Val Acc=0.5914
Epoch 10: Train Loss=165.5831, Train Acc=0.5701, Val Loss=39.6227, Val Acc=0.5930


In [5]:
import zipfile
import struct

def read_zip_binary(path):
    ragged_array = []
    with zipfile.ZipFile(path, 'r') as zf:
        inner_path = path.split("/")[-1].split(".")[0]
        with zf.open(f'{inner_path}.bin', 'r') as r:
            while True:
                size_bytes = r.read(4)
                if not size_bytes:
                    break
                sub_array_size = struct.unpack('i', size_bytes)[0]
                sub_array = list(struct.unpack(f'{sub_array_size}h', r.read(sub_array_size * 2)))
                ragged_array.append(sub_array)
    return ragged_array

# Read test data from zip
X_test_raw = read_zip_binary("../1.2_modeling_and_tuning/data/X_test.zip")
print("Loaded test samples:", len(X_test_raw))

Loaded test samples: 2649


In [6]:
X_test_stft = stft_layer(X_test_raw)
X_test_stft = X_test_stft.unsqueeze(1)  # Add channel dim for CNN (B, 1, Freq, Time)

In [7]:
import torch

cnn_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_stft = X_test_stft.to(device)

with torch.no_grad():
    outputs = cnn_model(X_test_stft)
    predictions = torch.argmax(outputs, dim=1).cpu().numpy()

In [9]:
import pandas as pd
df = pd.DataFrame(predictions, columns=["label"])
df.to_csv("augment.csv", index=False, header=False)
print("Saved predictions to augment.csv ✅")

Saved predictions to augment.csv ✅
